<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Arthropod_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Arthropod Taxonomy Orders Object Detection Dataset](https://www.kaggle.com/datasets/mistag/arthropod-taxonomy-orders-object-detection-dataset/)

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
# import utils_box as box
import matplotlib as mpl
import concurrent.futures
from threading import Lock
import json, math, random, os, pprint
from matplotlib import pyplot as plt
AUTO = tf.data.AUTOTUNE
pp = pprint.PrettyPrinter()

# from tensorflow_models.vision import box_ops as boxutils

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/'
!kaggle datasets download -d mistag/arthropod-taxonomy-orders-object-detection-dataset
!unzip *.zip && rm *.zip

In [5]:
images_per_shard = 481
target_width = 1024

CLASSES = ['Lepidoptera', 'Hymenoptera', 'Hemiptera', 'Odonata', 'Diptera', 'Araneae', 'Coleoptera']

RAW_CLASSES = CLASSES + ['_truncated', '_blurred', '_occluded']

In [57]:
class Progress:
    """Text mode progress bar.
    Usage:
            p = Progress(30)
            p.step()
            p.step()
            p.step(start=True) # to restart form 0%
    The progress bar displays a new header at each restart."""
    def __init__(self, maxi, size=100, msg=""):
        """
        :param maxi: the number of steps required to reach 100%
        :param size: the number of characters taken on the screen by the progress bar
        :param msg: the message displayed in the header of the progress bat
        """
        self.maxi = maxi
        self.p = self.__start_progress(maxi)()  # () to get the iterator from the generator
        self.header_printed = False
        self.msg = msg
        self.size = size
        self.lock = Lock()

    def step(self, reset=False):
        with self.lock:
            if reset:
                self.__init__(self.maxi, self.size, self.msg)
            if not self.header_printed:
                self.__print_header()
            next(self.p)

    def __print_header(self):
        print()
        format_string = "0%{: ^" + str(self.size - 6) + "}100%"
        print(format_string.format(self.msg))
        self.header_printed = True

    def __start_progress(self, maxi):
        def print_progress():
            # Bresenham's algorithm. Yields the number of dots printed.
            # This will always print 100 dots in max invocations.
            dx = maxi
            dy = self.size
            d = dy - dx
            for x in range(maxi):
                k = 0
                while d >= 0:
                    print('=', end="", flush=True)
                    k += 1
                    d -= dx
                d += dy
                yield k

        return print_progress

    
def no_decorations(ax):
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    

def display_detections(images, offsets, resizes, detections, classnames, ground_truth_boxes=[]):
    # scale and offset the detected boxes back to original image coordinates
    boxes   = [[ (x,y,w,h)  for _, x, y, w, h, score, klass in detection_list] for detection_list in detections]
    boxes   = [[ (x-ofs[1], y-ofs[0], w, h) for x,y,w,h in boxlist ] for boxlist, ofs in zip(boxes, offsets)]
    boxes   = [[ (x*rsz, y*rsz, w*rsz, h*rsz) for x,y,w,h in boxlist ] for boxlist, rsz in zip(boxes, resizes)]
    classes = [[ int(klass) for _, x, y, w, h, score, klass in detection_list] for detection_list in detections]
    scores  = [[ score      for _, x, y, w, h, score, klass in detection_list] for detection_list in detections]
    display_with_boxes(images, boxes, classes, scores, classnames, ground_truth_boxes)
    
    
# images, boxes and classes must have the same number of elements
# scores can be en empty list []. If it is not empty, it must also
# have the same number of elements.
# classnames is the list of possible classes (strings)
def display_with_boxes(images, boxes, classes, scores, classnames, ground_truth_boxes=[]):
    N = len(images)
    sqrtN = int(math.ceil(math.sqrt(N)))
    aspect = sum([im.shape[1]/im.shape[0] for im in images])/len(images) # mean aspect ratio of images
    fig = plt.figure(figsize=(15,15/aspect), frameon=False)
    
    for k in range(N):
        ax = plt.subplot(sqrtN, sqrtN, k+1)
        no_decorations(ax)
        plt.imshow(images[k])
        
        if ground_truth_boxes:
            for box in ground_truth_boxes[k]:
                x, y, w, h = (box[0], box[1], box[2]-box[0], box[3]-box[1]) # convert x1 y1 x2 y2 into xywh
                #x, y, w, h = (box[0], box[1], box[2], box[3])
                rect = mpl.patches.Rectangle((x, y),w,h,linewidth=4,edgecolor='#FFFFFFA0',facecolor='none')
                ax.add_patch(rect)

        for i, (box, klass) in enumerate(zip(boxes[k], classes[k])):
            x, y, w, h = (box[0], box[1], box[2]-box[0], box[3]-box[1]) # convert x1 y1 x2 y2 into xywh
            #x, y, w, h = (box[0], box[1], box[2], box[3])
            #label = classnames[klass-1] # predicted classes are 1-based
            label = classnames[klass]
            if scores:
                label += ' ' + str(int(scores[k][i]*100)) + '%' 
            rect = mpl.patches.Rectangle((x, y),w,h,linewidth=4,edgecolor='#00000080',facecolor='none')
            ax.add_patch(rect)
            rect = mpl.patches.Rectangle((x, y),w,h,linewidth=2,edgecolor='#FFFF00FF',facecolor='none')
            ax.add_patch(rect)
            plt.text(x, y, label, size=16, ha="left", va="top", color='#FFFF00FF',
                     bbox=dict(boxstyle="round", ec='#00000080', fc='#0000004E', linewidth=3) )
            plt.text(x, y, label, size=16, ha="left", va="top", color='#FFFF00FF',
                     bbox=dict(boxstyle="round", ec='#FFFF00FF', fc='#0000004E', linewidth=1.5) )
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.02, hspace=0.02)
    plt.show()

In [93]:
# load all the metadata
DATA_PATH = '/content/ArTaxOr/Araneae'
json_filename_pattern = '/content/ArTaxOr/Araneae/annotations/*.json'
jpeg_filename_pattern = '/content/ArTaxOr/Araneae/*.jpg'

def load_json(filename, p):
    p.step()
    with tf.io.gfile.GFile(filename, 'r') as f:
        return json.load(f)
    
def filename_key(filename):
    path, filename = os.path.split(filename)
    dirname = os.path.split(path)[1]
    return filename
    # return os.path.join(dirname, filename)
    
def load_metadata(filename_pattern, jpeg_filename_pattern):
    print("Scanning directory...", end=' ')
    json_filenames = tf.io.gfile.glob(json_filename_pattern)
    jpeg_filenames = tf.io.gfile.glob(jpeg_filename_pattern)
    print(f"found {len(json_filenames)} metadata files and {len(jpeg_filenames)} image files.")
    print("Loading metadata")
    p = Progress(len(json_filenames))
    with concurrent.futures.ThreadPoolExecutor() as exe:
        data = exe.map(lambda x: load_json(x,p), json_filenames)
    # data as a dictionary for easier cross-referencing
    data = {filename_key(d['asset']['path']):d for d in data}
    return data, jpeg_filenames

RAW_METADATA, JPEG_FILENAMES = load_metadata(json_filename_pattern, jpeg_filename_pattern)

Scanning directory... found 2418 metadata files and 2418 image files.
Loading metadata

0%                                                                                              100%

In [95]:
name = JPEG_FILENAMES[0]
RAW_METADATA[name[-16:]]

{'asset': {'format': 'jpg',
  'id': '1a34363b7db2be18a6c282c223c41bf2',
  'name': '4bf03aa7b9c1.jpg',
  'path': 'file:F:/ArTaxOr/Araneae/4bf03aa7b9c1.jpg',
  'size': {'width': 2048, 'height': 1533},
  'state': 2,
  'type': 1},
 'regions': [{'id': 'XRBvH-p4D',
   'type': 'RECTANGLE',
   'tags': ['Araneae'],
   'boundingBox': {'height': 1138.0028735632184,
    'width': 844.15770609319,
    'left': 567.9713261648745,
    'top': 154.18103448275863},
   'points': [{'x': 567.9713261648745, 'y': 154.18103448275863},
    {'x': 1412.1290322580646, 'y': 154.18103448275863},
    {'x': 1412.1290322580646, 'y': 1292.183908045977},
    {'x': 567.9713261648745, 'y': 1292.183908045977}]}],
 'version': '2.1.0'}

In [96]:
# Check that there is a model garden safe way of computing image ids.
# In the end, this does not matter. The ids are never used, but they must still be processed,
# otherwise the Model Garden data loader will crash on them.

# copied from Model Garden code
# = official/vision/beta/dataloaders/utils process_source_id()

def model_garden__str_to_int64(s):
    return tf.strings.to_number(s, tf.int64).numpy()

def compute_id_bytestring(s):
    
    computed_id = (int('0x' + s[:16], 16) ^ int('0x' + s[16:], 16)) & 0x0FFFFFFFFFFFFFFF
    return str(computed_id).encode('utf-8')

raw_ids = []
cnv_ids = []
cnv_cnv_ids = []

for i, key in enumerate(RAW_METADATA):
    iid = RAW_METADATA[key]['asset']['id']
    raw_ids.append(iid)
    iid = compute_id_bytestring(iid)
    #print("computed:", iid)
    cnv_ids.append(iid)
    iid = model_garden__str_to_int64(iid)
    cnv_cnv_ids.append(iid)

print(f"Original ids: {len(raw_ids)} Uniques:{len(set(raw_ids))} Collisions:{len(raw_ids) - len(set(raw_ids))}")
print(f"Original ids: {len(cnv_ids)} Uniques:{len(set(cnv_ids))} Collisions:{len(raw_ids) - len(set(cnv_ids))}")
print(f"Computed ids as converted by Model Garden:{len(cnv_cnv_ids)}, Uniques: {len(set(cnv_cnv_ids))}, Collisions: {len(raw_ids) - len(set(cnv_cnv_ids))}")

Original ids: 2418 Uniques:2418 Collisions:0
Original ids: 2418 Uniques:2418 Collisions:0
Computed ids as converted by Model Garden:2418, Uniques: 2418, Collisions: 0


In [ ]:
# collect weird aspec ratio images and very large images
oddities = [d['asset']['path'] for k,d in RAW_METADATA.items() if d['asset']['size']['width']>5400]
oddities += [d['asset']['path'] for k,d in RAW_METADATA.items() if d['asset']['size']['height']>5000]
oddities2 = [d['asset']['path'] for k,d in RAW_METADATA.items() if d['asset']['size']['height']/d['asset']['size']['width']>1.9]
oddities2 += [d['asset']['path'] for k,d in RAW_METADATA.items() if d['asset']['size']['width']/d['asset']['size']['height']>2.0]

nice_pics = ['/ArTaxOr/Lepidoptera/e7d7b4678088.jpg','/ArTaxOr/Lepidoptera/e74f298859ff.jpg','/ArTaxOr/Coleoptera/92c9a15e7362.jpg',
             '/ArTaxOr/Coleoptera/a1824522fddc.jpg','/ArTaxOr/Hymenoptera/7188c0cc8c9d.jpg','/ArTaxOr/Lepidoptera/b7197aead30b.jpg',
             '/ArTaxOr/Lepidoptera/dfc9ece476e6.jpg','/ArTaxOr/Araneae/a1488eb130e3.jpg','/ArTaxOr/Coleoptera/39c0eabccc41.jpg']
alt_pics = ['/ArTaxOr/Araneae/3c6491416c3f.jpg','/ArTaxOr/Araneae/81ff08857d15.jpg',
            '/ArTaxOr/Hymenoptera/f8f10bc28f5b.jpg','/ArTaxOr/Lepidoptera/e314c31efafd.jpg']

#filenames = tf.io.gfile.glob('../input/arthropod-taxonomy-orders-object-detection-dataset/ArTaxOr/*/*jpg')
#filenames = nice_pics
filenames = oddities+oddities2
#filenames = alt_pics

# display a couple of images from the raw_metadata with their bounding boxes
images = []
bboxes = []
taglists = []
random.shuffle(filenames)
for filename in filenames[:4]:
    print(DATA_PATH, filename_key(filename), filename)
    filepath = os.path.join(DATA_PATH, filename_key(filename))
    d = RAW_METADATA[filename_key(filename)]
    images.append(tf.image.decode_jpeg(tf.io.read_file(filepath)))
    bbxs = [region['boundingBox'] for region in d['regions']]
    # xywh to yxyx conversion
    bbxs = [[box['left'],box['top'], box['left']+box['width'], box['top']+box['height']] for box in bbxs]
    #tags = [region['tags'] for region in d['regions']] # all tags
    tags = [region['tags'][0] for region in d['regions']] # first tag only
    tags = [CLASSES.index(t) for t in tags] 
    bboxes.append(bbxs)
    taglists.append(tags)

display_with_boxes(images, bboxes, taglists, None, CLASSES, ground_truth_boxes=[])